# Competição FGV Quant e FGV Tech

## Descrição do Problema

A empresa quer desenvolver uma ferramenta de sugestão de preços de automóveis. O seu trabalho é criar um modelo preditivo usando como base os dados já disponíveis pela empresa.

## Métrica de Avaliação

Todos os modelos serão avaliados através do R2 entre o valor predito/valor estimado do modelo e o valor real dos dados do conjunto cars_test.

A pontuação R2 é o coeficiente de determinação do modelo

## Formato de Submissão

Submeta o arquivo em formato .csv somente com as colunas "id" e "selling_price", exemplo:

id,selling_price \
1,100000 \
2,1260000 \
3,900000

---

# Tentativa 3 - Ideias

- Normalizar todos os dados
    - Usar normalização one hot encoder em dados qualitativos?
    - Testar MinMaxScaler e StandardScaler no dados quant

1. Linearização L1

2. Linearização L2

3. Rodar Random Forest

4. Rodar Isolation Forest

# Analise dos dados

## Leitura dos dados

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
plt.style.use('dark_background')

train = pd.read_csv('data/cars_train.csv')
test = pd.read_csv('data/cars_test.csv')

In [2]:
train.head()

,id,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,0,Maruti 800 EX 5 Speed,2006,65000,120000,Petrol,Individual,Manual,Second Owner,16.1 kmpl,796 CC,37 bhp,59Nm@ 2500rpm,4.0
1,1,Maruti Baleno Delta 1.2,2017,550000,46000,Petrol,Individual,Manual,First Owner,21.4 kmpl,1197 CC,83.1 bhp,115Nm@ 4000rpm,5.0
2,2,Toyota Innova 2.5 G (Diesel) 7 Seater BS IV,2012,730000,210000,Diesel,Individual,Manual,First Owner,12.99 kmpl,2494 CC,100 bhp,200Nm@ 1400-3400rpm,7.0
3,3,Maruti Alto LXi,2010,135000,95000,Petrol,Individual,Manual,Third Owner,19.7 kmpl,796 CC,46.3 bhp,62Nm@ 3000rpm,5.0
4,4,Ford Ecosport 1.5 DV5 MT Trend,2016,515000,68609,Diesel,Dealer,Manual,First Owner,22.7 kmpl,1498 CC,89.84 bhp,204Nm@ 2000-2750rpm,5.0


In [3]:
test.head()

,id,name,year,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,0,Maruti Swift Dzire VXI 1.2,2018,25000,Petrol,Individual,Manual,First Owner,21.21 kmpl,1197 CC,81.80 bhp,113Nm@ 4200rpm,5.0
1,1,Hyundai i20 1.4 Asta,2017,43000,Diesel,Dealer,Manual,First Owner,22.54 kmpl,1396 CC,88.73 bhp,219.7Nm@ 1500-2750rpm,5.0
2,2,Maruti Alto 800 LXI,2014,15000,Petrol,Individual,Manual,Second Owner,22.74 kmpl,796 CC,47.3 bhp,69Nm@ 3500rpm,5.0
3,3,Toyota Fortuner 4x4 AT,2017,60000,Diesel,Dealer,Automatic,Second Owner,12.55 kmpl,2982 CC,168.5 bhp,360Nm@ 1400-3200rpm,7.0
4,4,Honda Amaze V CVT Petrol BSIV,2019,7032,Petrol,Trustmark Dealer,Automatic,First Owner,19.0 kmpl,1199 CC,88.76 bhp,110Nm@ 4800rpm,5.0


## Tratamento Inicial dos Dados de Treino

In [4]:
# Removendo os nulls do train
train = train.dropna()

# Transformando as colunas Mileage, Engine e Max_power em int
train['mileage'] = train.loc[:,'mileage'].str.replace(r'\D', '').astype(int)
train['engine'] = train.loc[:,'engine'].str.replace(r'\D', '').astype(int)
train['max_power'] = train.loc[:,'max_power'].str.replace(r'\D', '').astype(int)

# Adicionando uma coluna para a marca do carro e outra para o modelo
train['marca'] = train.loc[:,'name'].str.split(' ').str[0]
train['model'] = train.loc[:,'name'].str.split(' ').str[0] + " " + train['name'].str.split(' ').str[1]

train.head()

,id,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,marca,model
0,0,Maruti 800 EX 5 Speed,2006,65000,120000,Petrol,Individual,Manual,Second Owner,161,796,37,59Nm@ 2500rpm,4.0,Maruti,Maruti 800
1,1,Maruti Baleno Delta 1.2,2017,550000,46000,Petrol,Individual,Manual,First Owner,214,1197,831,115Nm@ 4000rpm,5.0,Maruti,Maruti Baleno
2,2,Toyota Innova 2.5 G (Diesel) 7 Seater BS IV,2012,730000,210000,Diesel,Individual,Manual,First Owner,1299,2494,100,200Nm@ 1400-3400rpm,7.0,Toyota,Toyota Innova
3,3,Maruti Alto LXi,2010,135000,95000,Petrol,Individual,Manual,Third Owner,197,796,463,62Nm@ 3000rpm,5.0,Maruti,Maruti Alto
4,4,Ford Ecosport 1.5 DV5 MT Trend,2016,515000,68609,Diesel,Dealer,Manual,First Owner,227,1498,8984,204Nm@ 2000-2750rpm,5.0,Ford,Ford Ecosport


### Análise dos dados qualitativos

In [5]:
train_quali = train.loc[:,['name','fuel', 'seller_type', 'transmission', 'owner', 'marca', 'model']]
train_quali.head()

,name,fuel,seller_type,transmission,owner,marca,model
0,Maruti 800 EX 5 Speed,Petrol,Individual,Manual,Second Owner,Maruti,Maruti 800
1,Maruti Baleno Delta 1.2,Petrol,Individual,Manual,First Owner,Maruti,Maruti Baleno
2,Toyota Innova 2.5 G (Diesel) 7 Seater BS IV,Diesel,Individual,Manual,First Owner,Toyota,Toyota Innova
3,Maruti Alto LXi,Petrol,Individual,Manual,Third Owner,Maruti,Maruti Alto
4,Ford Ecosport 1.5 DV5 MT Trend,Diesel,Dealer,Manual,First Owner,Ford,Ford Ecosport


In [6]:
# Removendo as colunas Id, name, torque, model
dummies_train = pd.get_dummies(train_quali)
dummies_train.head()

,name_Ambassador Classic 2000 DSZ AC PS,name_Ambassador Grand 1500 DSZ BSIII,name_Ashok Leyland Stile LE,name_Audi A3 35 TDI Premium Plus,name_Audi A3 40 TFSI Premium,name_Audi A4 1.8 TFSI,name_Audi A4 2.0 TDI,name_Audi A4 2.0 TDI 177 Bhp Premium Plus,name_Audi A4 35 TDI Premium Plus,name_Audi A6 2.0 TDI,...,model_Volkswagen GTI,model_Volkswagen Jetta,model_Volkswagen Passat,model_Volkswagen Polo,model_Volkswagen Vento,model_Volvo S60,model_Volvo S90,model_Volvo V40,model_Volvo XC40,model_Volvo XC90
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Análise dos dados quantitativos

In [ ]:
train_quant = train.copy().drop(columns=['id', 'name', 'torque', 'model'])
train_quant = train_quant.drop(columns=['fuel', 'seller_type', 'transmission', 'owner', 'marca'])
train_quant.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler_train = MinMaxScaler()
train_quant = scaler_train.fit_transform(train_quant.drop(columns='selling_price'))
train_quant = pd.DataFrame(train_quant)
train_quant.columns = ['year', 'km_driven', 'mileage', 'engine', 'max_power', 'seats']
train_quant.head()

----

## Tratamento Inicial dos Dados de Teste

In [ ]:
# Transformando as colunas Mileage, Engine e Max_power em int
test['mileage'] = test.loc[:,'mileage'].str.replace(r'\D', '').astype(int)
test['engine'] = test.loc[:,'engine'].str.replace(r'\D', '').astype(int)
test['max_power'] = test.loc[:,'max_power'].str.replace(r'\D', '').astype(int)

# Adicionando uma coluna para a marca do carro e outra para o modelo
test['marca'] = test.loc[:,'name'].str.split(' ').str[0]
test['model'] = test.loc[:,'name'].str.split(' ').str[0] + " " + train['name'].str.split(' ').str[1]


test = test.drop(columns = ['id','name','torque'])
test.head()

In [ ]:
test_quali = test.loc[:,['fuel', 'seller_type', 'transmission', 'owner']]
test_quali.head()

dummies_test = pd.get_dummies(test_quali)
dummies_test.head()

In [ ]:
test_quant = test.copy().drop(columns=['model'])
test_quant = test_quant.drop(columns=['fuel', 'seller_type', 'transmission', 'owner', 'marca'])
test_quant.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler_test = MinMaxScaler()
test_quant = scaler_test.fit_transform(test_quant)
test_quant = pd.DataFrame(test_quant)
test_quant.columns = ['year', 'km_driven', 'mileage', 'engine', 'max_power', 'seats']
test_quant.head()

----

# Modelagem

### Linear regression

In [ ]:
train_lr = pd.concat([train_quant, dummies_train], axis='columns')
train_lr.head()

In [ ]:
test_lr = pd.concat([test_quant, dummies_test], axis='columns')
test_lr.head()

In [ ]:
# Colunas iguais no Treino e Teste
cols_equal = []
for i in range(len(train_lr.columns)):
    for j in range(len(test_lr.columns)):
        if train_lr.columns[i] == test_lr.columns[j]:
            cols_equal.append(train_lr.columns[i])
            
# Colunas para dropar
cols_to_drop = train_lr.copy().drop(columns = cols_equal)
cols_to_drop

# 'marca_Ashok', 'marca_Daewoo', 'marca_Isuzu', 'marca_Land', 'marca_Opel'

In [ ]:
y = train_lr # previsor
x = train.drop(columns=["selling_price"]) # variáveis características
x.shape

In [ ]:
# Calculei os VIFs(Variance inflation factor) para retirar o efeito da multicolinearidade, excluindo as colunas com VIFs maiores do que 5

from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculatevif(X, thresh=5.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping '' + X.iloc[:, variables].columns[maxloc] + '' at index: ' + str(maxloc))
            del variables[maxloc]
            dropped = True

    print('Remaining variables:')
    print(X.columns[variables])
    return X.iloc[:, variables]

In [ ]:
X = calculatevif(x)

In [ ]:
X.shape, y.shape

In [ ]:
# Separar a base em treino e teste para validaçao

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 10)

# Criando o modelo

import statsmodels.api as sm

model = sm.OLS(y_train, sm.add_constant(X_train)).fit()
model.summary()

In [ ]:
# Removendo as colunas que possuem P-value > 0.05
#Backward Elimination
cols = list(X_train.columns)
pmax = 1
while (len(cols)>0):
    p = []
    X_1 = X_train[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y_train,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax > 0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)
print()
print(p.max(), p.idxmax(), p.max()<0.05)

In [ ]:
# Aplicando o modelo na nova base

model = sm.OLS(y_train, X_train[selected_features_BE]).fit()
model.summary()

## Análise de Resíduos

In [ ]:
plt.figure(figsize = (10,7))
plt.scatter(y_train,model.resid.values)
plt.axhline(0.05)
plt.axhline(-0.05)
plt.xlabel('fitted value')
plt.ylabel('residual')
plt.show()

In [ ]:
import scipy as sp
fig, ax = plt.subplots(figsize=(10,7))
sp.stats.probplot(model.resid.values, plot=ax, fit=True)

## Validação do Modelo

In [ ]:
# Fazendo a predição
predictions = model.predict(X_test[selected_features_BE])
model.rsquared

In [ ]:
# Criando um DataFrame com o preço e suas prediçoes

df = pd.DataFrame({'selling_price':y_test, 'Resultado':predictions})

In [ ]:
# Plotando o Grafico do preço em relaçao às prediçoes
import matplotlib.lines as mlines

fig, ax = plt.subplots()
sns.regplot(x="selling_price", y="Resultado", data=df);
sns.scatterplot(x='selling_price',y='Resultado', data=df)
line = mlines.Line2D([0, 1], [0, 1], color='red')
transform = ax.transAxes
line.set_transform(transform)
ax.add_line(line)
plt.show()

In [ ]:
from scipy import stats

slope, intercept, r_value, p_value, std_err = stats.linregress(df['selling_price'],predictions)
print ("r-squared:", r_value**2)

---

# Criando o CSV para o Kaggle

In [ ]:
predictions_test = model.predict(test_lr[selected_features_BE])
predictions_test

In [ ]:
df_test = predictions_test.to_frame(name='selling_price')
df_test = df_test.rename_axis('id').reset_index()
df_test

In [ ]:
'''
Terceiro deploy = 20h00 do dia 04/06/2021
'''
df_test.to_csv('Teste_3-Hugo_Azevedo.csv', index=False)